In [64]:
import numpy as np
from scipy.special import factorial2 as fact2
from scipy.special import erf

#@jit(nopython=True)
class BasisFunction:
    ''' A class that contains all our basis function data
            Attributes:
            origin: array/list containing the coordinates of the Gaussian origin
            shell:  tuple of angular momentum
            exps:   list of primitive Gaussian exponents
            coefs:  list of primitive Gaussian coefficients
            norm:   list of normalization factors for Gaussian primitives
    '''
    
    
    def __init__(self,origin=[0.0,0.0,0.0],shell=(0,0,0),exps=[],coefs=[]): 
        self.origin = np.asarray(origin)
        self.shell = shell
        self.exps = exps
        self.coefs = coefs
        self.norm = None
        self.normalize()
        
    
    def normalize(self):
        ''' Routine to normalize the basis functions, in case they
                    do not integrate to unity.
                '''
        l,m,n = self.shell
        L = l+m+n
        # self.norm is a list of length equal to number primitives 
        # normalize primitives first (PGBFs)
        self.norm = np.sqrt(np.power(2,2*(l+m+n)+1.5)*\
                            np.power(self.exps,l+m+n+1.5)/fact2(2*l-1)/fact2(2*m-1)/fact2(2*n-1)/np.power(np.pi,1.5))
        # now normalize the contracted basis functions (CGBFs) # Eq. 1.44 of Valeev integral whitepaper
        prefactor = np.power(np.pi,1.5)*\
                fact2(2*l - 1)*fact2(2*m - 1)*fact2(2*n - 1)/np.power(2.0,L)
        N = 0.0
        num_exps = len(self.exps) 
        for ia in range(num_exps):
            for ib in range(num_exps):
                N += self.norm[ia]*self.norm[ib]*self.coefs[ia]*self.coefs[ib]/np.power(self.exps[ia] + self.exps[ib],L+1.5)
        N *= prefactor
        N = np.power(N,-0.5)
        for ia in range(num_exps):
            self.coefs[ia] *= N
        
   
    def basisfcn_matrix(self,xdom,ydom,zdom):
        ''' Method that returns a 3-dimensional matrix corresponding to the 
            3D basis function evaluated in space on over a domain (xdom,ydom,zdom)
        '''
        l,m,n = self.shell
        xi = self.exps  
        G = np.zeros((len(xdom),len(ydom),len(zdom)))
        for i in range(len(xi)):
            gauss_x = xdom**l*np.exp(-xi[i]*(xdom-self.origin[0])**2)
            gauss_y = ydom**m*np.exp(-xi[i]*(ydom-self.origin[1])**2)
            gauss_z = zdom**n*np.exp(-xi[i]*(zdom-self.origin[2])**2)
            Temp = np.kron(gauss_z,np.kron(gauss_y,gauss_x))
            G += self.coefs[i]*np.reshape(Temp,[len(xdom),len(ydom),len(zdom)])
            
        return G
    
            
            
 #@njit
def E(i,j,t,Qx,a,b):
    ''' Recursive definition of Hermite Gaussian coefficients.
            Returns a float.
            a: orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b: orbital exponent on Gaussian 'b' (e.g. beta in the text)
            i,j: orbital angular momentum number on Gaussian 'a' and 'b'
            t: number nodes in Hermite (depends on type of integral,
               e.g. always zero for overlap integrals)
            Qx: distance between origins of Gaussian 'a' and 'b'
    '''
    p=a+b
    q = a*b/p
    if (t < 0) or (t > (i + j)): # out of bounds for t
        return 0.0
    elif i == j == t == 0: # base case
        return np.exp(-q*Qx*Qx) # K_AB 
    elif j == 0: # decrement index i
        return (1/(2*p))*E(i-1,j,t-1,Qx,a,b) -  (q*Qx/a)*E(i-1,j,t,Qx,a,b) +  (t+1)*E(i-1,j,t+1,Qx,a,b)
    else: # decrement index j
        return (1/(2*p))*E(i,j-1,t-1,Qx,a,b) +  (q*Qx/b)*E(i,j-1,t,Qx,a,b) +  (t+1)*E(i,j-1,t+1,Qx,a,b)

def overlap(a,lmn1,A,b,lmn2,B):
    ''' Evaluates overlap integral between two Gaussians
            Returns a float.
            a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
            lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                  for Gaussian 'a'
            lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
            A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
            B:    list containing origin of Gaussian 'b'
    '''
    l1,m1,n1 = lmn1 # shell angular momentum on Gaussian 'a' 
    l2,m2,n2 = lmn2 # shell angular momentum on Gaussian 'b' 
    S1 = E(l1,l2,0,A[0]-B[0],a,b) # X
    S2 = E(m1,m2,0,A[1]-B[1],a,b) # Y
    S3 = E(n1,n2,0,A[2]-B[2],a,b) # Z
    return S1*S2*S3*np.power(np.pi/(a+b),1.5)


def kinetic(a,lmn1,A,b,lmn2,B):
    ''' Evaluates kinetic energy integral between two Gaussians
            Returns a float.
            a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
            b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
            lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                  for Gaussian 'a'
            lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
            A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
            B:    list containing origin of Gaussian 'b'
    '''
    
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    
    term0 = b*(2*(l2+m2+n2)+3)*overlap(a,(l1,m1,n1),A,b,(l2,m2,n2),B)
    
    term1 = -2*np.power(b,2)*(overlap(a,(l1,m1,n1),A,b,(l2+2,m2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2+2,n2),B) +
                            overlap(a,(l1,m1,n1),A,b,(l2,m2,n2+2),B))
    
    term2 = -0.5*(l2*(l2-1)*overlap(a,(l1,m1,n1),A,b,(l2-2,m2,n2),B) +
                  m2*(m2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2-2,n2),B) +
                  n2*(n2-1)*overlap(a,(l1,m1,n1),A,b,(l2,m2,n2-2),B)) 
    
    return term0+term1+term2

#@jit(nopython=True)
def R(t,u,v,n,p,PCx,PCy,PCz,RPC):
    ''' Returns the Coulomb auxiliary Hermite integrals
            Returns a float.
            Arguments:
            t,u,v:   order of Coulomb Hermite derivative in x,y,z
                     (see defs in Helgaker and Taylor)
            n:       order of Boys function
            PCx,y,z: Cartesian vector distance between Gaussian
                     composite center P and nuclear center C
            RPC:     Distance between P and C
    '''
    T = p*RPC*RPC
    val = 0.0
    if t == u == v == 0:
        val += np.power(-2*p,n)*boys(float(n),T) 
    elif t == u == 0:
        if v > 1:
            val += (v-1)*R(t,u,v-2,n+1,p,PCx,PCy,PCz,RPC)
        val += PCz*R(t,u,v-1,n+1,p,PCx,PCy,PCz,RPC) 
    elif t == 0:
        if u > 1:
            val += (u-1)*R(t,u-2,v,n+1,p,PCx,PCy,PCz,RPC)
        val += PCy*R(t,u-1,v,n+1,p,PCx,PCy,PCz,RPC) 
    else:
        if t > 1:
            val += (t-1)*R(t-2,u,v,n+1,p,PCx,PCy,PCz,RPC)
        val += PCx*R(t-1,u,v,n+1,p,PCx,PCy,PCz,RPC) 
    return val

from scipy.special import hyp1f1
#@jit(nopython=True)
def boys(n,T):
    return hyp1f1(n+0.5,n+1.5,-T)/(2.0*n+1.0)

#@jit(nopython=True)
def gaussian_product_center(a,A,b,B): 
    return (a*A+b*B)/(a+b)


def nuclear_attraction(a,lmn1,A,b,lmn2,B,C):
    ''' Evaluates kinetic energy integral between two Gaussians
             Returns a float.
             a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
             b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
             lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
                   for Gaussian 'a'
             lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
             A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
             B:    list containing origin of Gaussian 'b'
             C:    list containing origin of nuclear center 'C'
    '''
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    p=a+b
    P = gaussian_product_center(a,A,b,B) # Gaussian composite center 
    RPC = np.linalg.norm(P-C)
    val = 0.0
    for t in range(l1+l2+1):
        for u in range(m1+m2+1):
            for v in range(n1+n2+1):
                 val += E(l1,l2,t,A[0]-B[0],a,b) * E(m1,m2,u,A[1]-B[1],a,b) * E(n1,n2,v,A[2]-B[2],a,b) * \
                    R(t,u,v,0,p,P[0]-C[0],P[1]-C[1],P[2]-C[2],RPC)
    val *= 2*np.pi/p 
    return val

#@jit(nopython=True)
def electron_repulsion(a,lmn1,A,b,lmn2,B,c,lmn3,C,d,lmn4,D): 
    ''' Evaluates kinetic energy integral between two Gaussians
         Returns a float.
         a,b,c,d:   orbital exponent on Gaussian 'a','b','c','d'
         lmn1,lmn2
         lmn3,lmn4: int tuple containing orbital angular momentum
                    for Gaussian 'a','b','c','d', respectively
         A,B,C,D:   list containing origin of Gaussian 'a','b','c','d'
    '''
    l1,m1,n1 = lmn1
    l2,m2,n2 = lmn2
    l3,m3,n3 = lmn3
    l4,m4,n4 = lmn4
    p = a+b # composite exponent for P (from Gaussians 'a' and 'b') 
    q = c+d # composite exponent for Q (from Gaussians 'c' and 'd') 
    alpha = p*q/(p+q)
    P = gaussian_product_center(a,A,b,B) # A and B composite center 
    Q = gaussian_product_center(c,C,d,D) # C and D composite center 
    RPQ = np.linalg.norm(P-Q)
    val = 0.0
    for t in range(l1+l2+1):
        for u in range(m1+m2+1):
            for v in range(n1+n2+1):
                for tau in range(l3+l4+1): 
                    for nu in range(m3+m4+1):
                        for phi in range(n3+n4+1):
                            val += E(l1,l2,t,A[0]-B[0],a,b) * \
                                    E(m1,m2,u,A[1]-B[1],a,b) * \
                                    E(n1,n2,v,A[2]-B[2],a,b) * \
                                    E(l3,l4,tau,C[0]-D[0],c,d) * \
                                    E(m3,m4,nu ,C[1]-D[1],c,d) * \
                                    E(n3,n4,phi,C[2]-D[2],c,d) * \
                                    np.power(-1,tau+nu+phi) * \
                                    R(t+tau,u+nu,v+phi,0,\
                                        alpha,P[0]-Q[0],P[1]-Q[1],P[2]-Q[2],RPQ)
    val *= 2*np.power(np.pi,2.5)/(p*q*np.sqrt(p+q)) 
    return val


def S(a,b):
    '''Evaluates overlap between two contracted Gaussians
           Returns float.
           Arguments:
           a: contracted Gaussian 'a', BasisFunction object
           b: contracted Gaussian 'b', BasisFunction object
    '''
    s = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            s += a.norm[ia]*b.norm[ib]*ca*cb*\
                 overlap(a.exps[ia],a.shell,a.origin,
                 b.exps[ib],b.shell,b.origin)
    return s


def T(a,b):
    '''Evaluates kinetic energy between two contracted Gaussians
           Returns float.
           Arguments:
           a: contracted Gaussian 'a', BasisFunction object
           b: contracted Gaussian 'b', BasisFunction object
    '''
    t = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            t += a.norm[ia]*b.norm[ib]*ca*cb*kinetic(a.exps[ia],a.shell,a.origin,b.exps[ib],b.shell,b.origin)
    return t


def V(a,b,C,Z):
    '''Evaluates overlap between two contracted Gaussians
            Returns float.
            Arguments:
            a: contracted Gaussian 'a', BasisFunction object
            b: contracted Gaussian 'b', BasisFunction object
            C: center of nucleus
    '''
    v = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            v += a.norm[ia]*b.norm[ib]*ca*cb*nuclear_attraction(a.exps[ia],a.shell,a.origin,b.exps[ib],b.shell,b.origin,C)
    return -v*Z


def ERI(a,b,c,d):
    '''Evaluates overlap between two contracted Gaussians
            Returns float.
            Arguments:
            a: contracted Gaussian 'a', BasisFunction object
            b: contracted Gaussian 'b', BasisFunction object
            c: contracted Gaussian 'b', BasisFunction object
            d: contracted Gaussian 'b', BasisFunction object
    '''
    eri = 0.0
    for ja, ca in enumerate(a.coefs):
        for jb, cb in enumerate(b.coefs):
            for jc, cc in enumerate(c.coefs):
                for jd, cd in enumerate(d.coefs):
                    eri += a.norm[ja]*b.norm[jb]*c.norm[jc]*d.norm[jd]*\
                           ca*cb*cc*cd*\
                           electron_repulsion(a.exps[ja],a.shell,a.origin,\
                                b.exps[jb],b.shell,b.origin,\
                                c.exps[jc],c.shell,c.origin,\
                                d.exps[jd],d.shell,d.origin)
    return eri

In [86]:
homo_eig = []
lumo_eig = []

with open('homo_lumo_eig.dat') as fid:
    lines = fid.readlines()
    ct = 1
    for line in lines:
        lsplit = line.strip().split()
        if len(lsplit) > 5:
            homo_eig.append(float(lsplit[6]))
            lumo_eig.append(float(lsplit[7]))

fid.close()

homo_eig = np.asarray(homo_eig)
lumo_eig = np.asarray(lumo_eig)

atom_charge = []
atom_id = []
atom_coord_A = [] # angstrom
atom_coord_B = [] # angstrom

ang_to_bohr = 1.88973;

delta = np.array([0,0,7])


with open('monomer_scf.inp') as fid:
    lines = fid.readlines()
    flag = False
    for line in lines:
        lsplit = line.strip().split()
        if len(lsplit) > 0:
            if lsplit[0] == 'c1':
                flag = True
            if flag:
                if len(lsplit) > 2:
                    atom_id.append(lsplit[0])
                    atom_charge.append(int(float(lsplit[1])))
                    
                    xA = float(lsplit[2])*ang_to_bohr
                    yA = float(lsplit[3])*ang_to_bohr
                    zA = float(lsplit[4])*ang_to_bohr
                    
                    xB = float(lsplit[2])*ang_to_bohr + delta[0]
                    yB = float(lsplit[3])*ang_to_bohr + delta[1]
                    zB = float(lsplit[4])*ang_to_bohr + delta[2]
                    
                    atom_coord_A.append(np.array([xA,yA,zA]))
                    atom_coord_B.append(np.array([xB,yB,zB]))
            else:
                continue
fid.close()

In [91]:
AA = np.asarray(atom_coord_A)
BB = np.asarray(atom_coord_B)

np.min(AA[:,2])
#np.min(BB[:,2])

-5.9792191038

In [66]:
def construct_orbitals(SHELL, XI, CM, atom_coordinates):
    orbs = []; Nat = len(SHELL);
    for i in range(Nat):
        for j in range(len(XI[i])):
            orbs.append(BasisFunction(origin=atom_coordinates[i][:],shell=SHELL[i][j],exps=XI[i][j],coefs=CM[i][j]))
    return orbs

In [101]:
def calculate_twobody_ints(orbs,vec,eig_v,tol):
    
    Norb = len(orbs)
    VVmat = np.zeros((Norb,Norb,Norb,Norb))
    
    if vec: # slower?
        num_twobody = int(Norb*(Norb+1)*(Norb**2+Norb+2)/8)
        VVmatvec = np.zeros(num_twobody)
        # Calculate only permutationally unique twobody integrals
        ct = 0
        for i in range(Norb):
            for j in range(i+1):
                for k in range(i+1):
                    if i == k: 
                        lmax = j+1 
                    else: 
                        lmax = k+1
                    for l in range(lmax):
                        VVmatvec[ct] = ERI(orbs[i],orbs[j],orbs[k],orbs[l]) # chemist notation
                        ct += 1
        # Insert unique integrals into 4-dimensional array
        for i in range(Norb):
            for j in range(Norb):
                if i > j: 
                    ij = i*(i+1)/2+j 
                else: 
                    ij = j*(j+1)/2 + i
                for k in range(Norb):
                    for l in range(Norb):
                        if k > l: 
                            kl = k*(k+1)/2+l 
                        else: 
                            kl = l*(l+1)/2 + k
                        if ij > kl: 
                            ijkl = ij*(ij+1)/2 + kl 
                        else: 
                            ijkl = kl*(kl+1)/2 + ij       
                        VVmat[i,k,j,l] = VVmatvec[int(ijkl)] # physics notation
        return VVmat,VVmatvec
    else:
        # Calculate twobody integrals
        for i in range(Norb):
            print('{}'.format(i+1))
            for j in range(i+1):
                ij = i*(i+1)/2 + j
                for k in range(Norb):
                    for l in range(k+1):
                        kl = k*(k+1)//2 + l
                        if ij >= kl:
                            if abs(eig_v[i]*eig_v[j]*eig_v[k]*eig_v[l]) > tol:
                                val = ERI(orbs[i],orbs[j],orbs[k],orbs[l])
                                VVmat[i,k,j,l] = val
                                VVmat[j,k,i,l] = val
                                VVmat[i,l,j,k] = val
                                VVmat[k,i,l,j] = val
                                VVmat[l,i,k,j] = val
                                VVmat[l,j,k,i] = val
                                VVmat[j,l,i,k] = val
                                VVmat[k,j,l,i] = val
        return VVmat

In [68]:
def bse_parse(basis,Z):
    import basis_set_exchange as bse
    BS = bse.get_basis(basis,elements=[Z],fmt='gamess_us',header=False)
    shell = []
    xi = []
    Cm = []
    BSS = BS.splitlines()[3:-1]
    for line in BSS:
        x = line.split()
        if x[0] == 'S' or x[0] == 's':
            SH = 'S'
            NGS = int(x[1])
            shell.append((0,0,0))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'P' or x[0] == 'p':
            SH = 'P'
            NGS = int(x[1])
            shell.append((1,0,0))
            shell.append((0,1,0))
            shell.append((0,0,1))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'D' or x[0] == 'd':
            SH = 'D'
            NGS = int(x[1])
            shell.append((2,0,0))
            shell.append((0,2,0))
            shell.append((0,0,2))
            shell.append((1,1,0))
            shell.append((0,1,1))
            shell.append((1,0,1))
            xi_temp = []
            Cm_temp = []
        elif x[0] == 'L' or x[0] == 'l':
            SH = 'L'
            NGS = int(x[1])
            shell.append((0,0,0))
            shell.append((1,0,0))
            shell.append((0,1,0))
            shell.append((0,0,1))
            xi_temp = []
            Cm_temp_s = []
            Cm_temp_p = []
        else:
            if SH != 'L':
                xi_temp.append(float(x[1]))
                Cm_temp.append(float(x[2]))
            else:
                xi_temp.append(float(x[1]))
                Cm_temp_s.append(float(x[2]))
                Cm_temp_p.append(float(x[3]))
            if len(xi_temp) == NGS:
                if SH != 'L':
                    if SH == 'S':
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                    elif SH == 'P':
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                    elif SH == 'D':
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        xi.append(xi_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                        Cm.append(Cm_temp)
                    else:
                        print('Unknown shell label')
                else:
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    xi.append(xi_temp)
                    Cm.append(Cm_temp_s)
                    Cm.append(Cm_temp_p)
                    Cm.append(Cm_temp_p)
                    Cm.append(Cm_temp_p)
    return shell, xi, Cm

def get_basis(basis,Z):
    SHELL = []
    XI = []
    CM = []
    for z in Z:
        shell,xi,Cm = bse_parse(basis,z)
        SHELL.append(shell)
        XI.append(xi)
        CM.append(Cm)
    return SHELL,XI,CM


In [69]:
# Get the basis set for the calculation
import itertools

basis = '3-21G'
SHELL, XI, CM = get_basis(basis,atom_charge)
Norb = len(list(itertools.chain.from_iterable(XI)))
XI_2 = list(itertools.chain.from_iterable(XI))
CM_2 = list(itertools.chain.from_iterable(CM))
SHELL_2 = list(itertools.chain.from_iterable(SHELL))

In [92]:
orbsA = construct_orbitals(SHELL, XI, CM, atom_coord_A)
orbsB = construct_orbitals(SHELL, XI, CM, atom_coord_B)

In [106]:
def twobody_A_B(orbs,homo_eig,lumo_eig,tol):
    
    Norb = len(orbs[0])
    
    EIG = [homo_eig, lumo_eig]
    
    VEC = []
    
    for at1 in range(2):
        for at2 in range(2):
            for at3 in range(2):
                for at4 in range(2):
                    
                    print('calculating {}{}{}{}...\n'.format(at1+1,at2+1,at3+1,at4+1))
            
                    O1 = orbs[at1]
                    O2 = orbs[at2]
                    O3 = orbs[at3]
                    O4 = orbs[at4]
                    
                    eig_vv_mat = np.zeros((len(EIG),len(EIG),len(EIG),len(EIG)))
                    
                    for p in range(len(EIG)):
                        for q in range(len(EIG)):
                            for r in range(len(EIG)):
                                for s in range(len(EIG)):

                                    print('pqrs: {}{}{}{}'.format(p+1,q+1,r+1,s+1))

                                    VVmat = np.zeros((Norb,Norb,Norb,Norb))

                                    temp = 0.0
                                    for alpha in range(Norb):
                                        print('alpha: {}'.format(alpha+1))
                                        for beta in range(Norb):
                                            for gamma in range(Norb):
                                                for delta in range(Norb):
                                                    pre = EIG[p][alpha]*EIG[q][beta]*EIG[r][gamma]*EIG[s][delta]
                                                    if np.abs(pre) > tol:
                                                        val = ERI(O1[alpha],O2[beta],O3[gamma],O4[delta])
                                                        temp += pre*val
                                                    else:
                                                        continue


                                    eig_vv_mat[p,q,r,s] = temp
                                    
                    VEC.append(eig_vv_mat)

    return VEC

In [107]:
VEC = twobody_A_B([orbsA,orbsB],homo_eig,lumo_eig,10**-4)

calculating 1111...

pqrs: 1111
alpha: 1
alpha: 2
alpha: 3
alpha: 4
alpha: 5
alpha: 6
alpha: 7
alpha: 8
alpha: 9
alpha: 10
alpha: 11
alpha: 12
alpha: 13
alpha: 14
alpha: 15
alpha: 16
alpha: 17
alpha: 18
alpha: 19
alpha: 20
alpha: 21
alpha: 22
alpha: 23
alpha: 24
alpha: 25
alpha: 26
alpha: 27
alpha: 28
alpha: 29
alpha: 30
alpha: 31


KeyboardInterrupt: 